### дз 2(2я часть - сравнение качества работы программы из 1го дз с ф-ей, выделяющей биграммы) 

### Генералова Софья 182

In [ ]:
!pip install sklearn

In [1]:
from sklearn.metrics import accuracy_score

### данные

примерно в пропорции 25% (125+ 125-) тестовые, 75% (375+ 375-) тренинг

с одного фильма - 1 полож отзыв, 1 отриц отзыв в тестовый дикт и так же в тренинговый

In [2]:
import re
from collections import Counter
import os
import json
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent
import random, time

In [3]:
import nltk

In [4]:
import pymorphy2

In [5]:
morph = pymorphy2.MorphAnalyzer()

### unofficial api kinopoisk
https://kinopoiskapiunofficial.tech/documentation/api/

In [21]:
genres = set([3, 19, 17, 16, 7, 6, 13, 22, 8, 23, 10, 11, 4, 2])


In [22]:
TOKEN = '2d63343e-b155-47b6-8d01-548bdd7f7902'

In [8]:
def get_list_of_movies(genre, page=1):
    data = requests.get(
        'https://kinopoiskapiunofficial.tech/api/v2.1/films/search-by-filters', 
        params={
            "genre": genre, #1,  # action
            "page":page,
        },
    
        headers={
            "X-API-KEY": TOKEN,
        }
    
    ).json()
    
            
    return data

### собирает айди

In [39]:
def get_ids(some_d, what, tr_p=True, test_p=True, tr_n=True, test_n=True, range_st=1):
    if what == 'f': # для фильма
        for film in some_d['films']:
            ids_set.add(film["filmId"])
    elif what == 'r': # для отзыва
        for rev in some_d['reviews']:
            if not (tr_p or test_p or tr_n or test_n): # если на этот фильм уже собрано 4 отзыва
                break
            if rev['reviewType'] == 'POSITIVE':
                if tr_p: # если еще нет полож отзыва для обучения
                    ids_dict_r[rev['reviewType']].add(rev["reviewId"])
                    tr_p = False
                elif test_p: # если еще нет полож отзыва для теста
                    test_ids_dict_r[rev['reviewType']].add(rev["reviewId"])
                    test_p = False
                    
            elif rev['reviewType'] == 'NEGATIVE':
                if tr_n: # если еще нет отриц отзыва для обучения
                    ids_dict_r[rev['reviewType']].add(rev["reviewId"])
                    tr_n = False
                elif test_n: # если еще нет отриц отзыва для теста
                    test_ids_dict_r[rev['reviewType']].add(rev["reviewId"])
                    test_n = False
        
        if tr_p or test_p or tr_n or test_n: # если какого-то типа отзывов не было
            manyRevs = some_d['pagesCount']
            if manyRevs: # чтобы не попался фильм у которого все поля null (id фильма 745552)
                if manyRevs > 1 and range_st < manyRevs: # если стр больше одной
                    range_st += 1
                    get_ids(get_rev_ids(some_d['filmId'], range_st), 'r', tr_p, test_p, tr_n, test_n, range_st)
                

In [40]:
ids_dict_r = {'NEGATIVE': set(), 'POSITIVE': set()} # dict for training
test_ids_dict_r = {'NEGATIVE': set(), 'POSITIVE': set()} # dict for testing

In [41]:
ids_set = set() # set for films ids

### достает список айди отзывов

In [12]:
def get_rev_ids(film_id, page=1):
    data = requests.get(
        'https://kinopoiskapiunofficial.tech/api/v1/reviews',
        params={
            "filmId": film_id,
            "page":page,
        },
    
        headers={
            "X-API-KEY": TOKEN,
        }
    
    ).json()
    
    return data

### достает текст отзыва по его айди

In [19]:
def get_rev(rev_id):
    data = requests.get(
        'https://kinopoiskapiunofficial.tech/api/v1/reviews/details', 
        params={
            "reviewId": rev_id,
        },
    
        headers={
            "X-API-KEY": TOKEN,
        }
    
    ).json()
    
    rev_text = data['reviewDescription']
    
    return rev_text

In [24]:
requests.get(
        'https://kinopoiskapiunofficial.tech/api/v2.1/films/search-by-filters', 
        params={
            "genre": 1, #1,  # action
            "page":1,
        },
    
        headers={
            "X-API-KEY": TOKEN,
        }
    
    ).json()['pagesCount']

5

### достать айди отзывов по айди фильмов

In [14]:
def g_r(ids_set):
    for film_id in ids_set: # распаковка отзывов на эти фильмы
        revs = get_rev_ids(film_id)
        get_ids(revs, 'r')
    ids_set = set() # убрать айди фильмов кот уже проверены
    return ids_set

### скачивание

In [42]:
for genre in genres:
    # проверка кол-ва отзывов
    if len(ids_dict_r['NEGATIVE']) < 375 or len(ids_dict_r['POSITIVE']) < 375 or len(test_ids_dict_r['NEGATIVE']) < 125 or len(test_ids_dict_r['POSITIVE']) < 125:
        list_of_m = get_list_of_movies(genre)
        if not list_of_m:
            print(genre)
        manyPages = list_of_m['pagesCount']
#         print(type(manyPages), manyPages)
        get_ids(list_of_m, 'f')
        ids_set = g_r(ids_set)
        
        if manyPages > 1:
            for i in range(2, manyPages+1):
                get_ids(get_list_of_movies(genre, i), 'f')
                ids_set = g_r(ids_set)
                    
        print('genre', genre, 'added')

genre 2 added
genre 3 added
genre 4 added
genre 6 added
genre 7 added
genre 8 added
genre 10 added
genre 11 added
genre 13 added
genre 16 added
genre 17 added
genre 19 added
genre 22 added


In [43]:
print(len(ids_dict_r['POSITIVE']))
print(len(ids_dict_r['NEGATIVE']))
print(len(test_ids_dict_r['POSITIVE']))
print(len(test_ids_dict_r['NEGATIVE']))

753
403
710
285


### видимо на некоторые фильмы вообще нет отрицательных отзывов, положительных собралось больше. поэтому убираю лишние айди:

In [44]:
test_ids_dict_r['POSITIVE'] = set(list(test_ids_dict_r['POSITIVE'])[:285])
ids_dict_r['POSITIVE'] = set(list(ids_dict_r['POSITIVE'])[:403])

### теперь все более менее одинаково

In [45]:
print(len(ids_dict_r['POSITIVE']))
print(len(ids_dict_r['NEGATIVE']))
print(len(test_ids_dict_r['POSITIVE']))
print(len(test_ids_dict_r['NEGATIVE']))

403
403
285
285


# добавляю выделение синт. групп

In [47]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [107]:
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

In [110]:
def find_syntax_phrases(text):
    tokens = []
    doc = Doc(text)
    doc.segment(segmenter) # tokenization
    doc.tag_morph(morph_tagger) # pos tagging
    doc.parse_syntax(syntax_parser) # syntax tagging
    h_id = None # переменная куда я буду записывать айди вершины для не
    ne = ''
    for i in range(len(doc.tokens)):
        r_t = doc.tokens[i].text
        
        if r_t == 'не':
            ne = r_t
            h_id = doc.tokens[i].head_id
            continue
        if h_id:
            # если айди элемента совп с айди вершины для не и часть речи подходит
            if doc.tokens[i].id == h_id and doc.tokens[i].pos in ['ADV', 'VERB', 'ADJ']:
                res = ne + ' ' + doc.tokens[i].text
                tokens.append(res)
                h_id = None
                ne = ''
            else:
                tokens.append(doc.tokens[i].text)
        else:
            tokens.append(doc.tokens[i].text)
    if ne != '': # если было не, но не было вершины
        tokens.append(ne)
  
    return tokens

### каунтер на основе токенизированных и лемматизированных отзывов

In [108]:
def most_com_w(c_type, ids):
    for rev_id in ids:
        rev_text = get_rev(rev_id)
        t = rev_text.lower()
        tokens = find_syntax_phrases(t) # выдел токенов в том числе синт групп
#         tokens = nltk.word_tokenize() # tokenization
        for t in tokens:
            if t.isalpha():
                norm_form = morph.parse(t)[0].normal_form # lemmatization
                c_type[norm_form]+=1
    return c_type

### по каунтеру для негативных и для позитивных отзывов

In [111]:
c_neg = most_com_w(Counter(), ids_dict_r['NEGATIVE'])

In [112]:
c_pos = most_com_w(Counter(), ids_dict_r['POSITIVE'])

словари из каунтеров

In [113]:
mc_n = dict(c_neg)# .most_common()

In [114]:
mc_p = dict(c_pos)

In [115]:
print(len(mc_n), len(mc_p))

15850 15913


множества из слов, отсекая частотности равные 1 или 2

In [116]:
mc_n_set = set(el for el in mc_n if mc_n[el] > 2)

In [117]:
mc_p_set = set(el for el in mc_p if mc_p[el] > 2)

In [118]:
print(len(mc_n_set), len(mc_p_set))

5395 5532


удаляю пересечения

In [119]:
del_set = mc_n_set & mc_p_set

In [120]:
mc_n_set -= del_set
mc_p_set -= del_set

In [121]:
print(len(mc_n_set), len(mc_p_set))

1795 1932


### на основе непересекающихся множеств с фильтром по частотности 2

In [122]:
def pos_or_neg_check(test):
    negcount = 0
    poscount = 0
    tokens = nltk.word_tokenize(test.lower())
#     print(tokens) # tokenization
    for t in tokens:
        if t.isalpha():
            norm_form = morph.parse(t)[0].normal_form
            if norm_form in mc_n_set:
                negcount += 1
#                 print(norm_form)
            elif norm_form in mc_p_set:
                poscount += 1
#                 print(norm_form)
    if negcount > poscount:
        
#         print('NEGATIVE')
#         print(negcount, poscount)
        return 'NEGATIVE'
    else:
#         print('POSITIVE')
#         print(negcount, poscount)
        return 'POSITIVE'

In [123]:
def accuracy_check(test_ids_dict_r):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходный язык
    for typ in test_ids_dict_r:
        for rev_id in test_ids_dict_r[typ]:
            rev_text = get_rev(rev_id) # получить текст отзыва по айди
            predicted_type = pos_or_neg_check(rev_text)
            results.append(predicted_type)
            gold.append(typ)
            
    print("RESULTS:")
    print("Test size: %d texts per negative review type" % len(test_ids_dict_r['NEGATIVE']))
    print("Test size: %d texts per positive review type" % len(test_ids_dict_r['POSITIVE']))
    print("Accuracy: %.4f" % accuracy_score(results, gold))

# с наташей:

In [124]:
accuracy_check(test_ids_dict_r)

RESULTS:
Test size: 285 texts per negative review type
Test size: 285 texts per positive review type
Accuracy: 0.7123


# предыдущий результат:

In [34]:
accuracy_check(test_ids_dict_r)

RESULTS:
Test size: 285 texts per negative review type
Test size: 285 texts per positive review type
Accuracy: 0.6877


# результаты:

было 0.6877, стало 0.7123
качество улучшилось, но не сильно